# NOTEBOOK EXPLANATION

This notebook is used to identify clusters in the 50plex experiment, map them back into the tissue and integrate the dataset with other technologies

# Load packages

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns

# Load the data

In [ ]:
re=pd.read_csv('U:/PROJECT_DirectRNA/directRNA/50plex_MAY/50_plex_decoded_20X/50plexV1_SAMPLE1_SPOTS_found_REVISED_26_JUNE_14CYCLES_GFAP.csv')

In [ ]:
re.columns=['ID','X','Y','gene']

In [ ]:
IN=range(1,50)
USEDNAMES=['Mbp','Slc17a7','Lamp5','Cd24a','Gad2','Cnr1','Pthlh','Crhbp','Slc32a1',
'Vip','Tbr1','Kcnip2','Crh','Cpne5','Rorb','Serpinf1','Gfap','Aldoc','Syt6','Mfge8','Cemip2','Pdgfra','Plp1',
'Sox10','Itpr2','Hexb','Anln','Bmp4','Ctps','Mrc1','Acta2','Apln','Flt1','Vtn','Ttr','Synpr','Npy',
'Sst','Foxj1','Pcp4','Calm2','Gad1','Pvalb','Penk','Rgs4','Nrn1','Tac2','Calb2','Slc6a1','Enc1'];
dictionary=dict(zip(IN,USEDNAMES))

In [ ]:
dictionary

In [ ]:
re['gene']=re['gene'].map(dictionary)

In [ ]:
re.to_csv('U:/PROJECT_DirectRNA/directRNA/50plex_reads.csv')

In [ ]:
data=pd.read_csv('F:\PROJECT_DirectRNA\directRNA\SUPPTABLE_individual_cell_expression.csv')

In [ ]:
data.columns

In [ ]:
metadata=data.iloc[:,[0,1,2,3,4,55]]
expdata=data.iloc[:,range(5,55)]

# Create AnnData object, filter and cluster

In [ ]:
adata = sc.AnnData(expdata)
adata.obs=metadata

In [ ]:
sc.pp.filter_cells(adata, min_genes=4)

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.neighbors(adata, n_neighbors=14, n_pcs=40)
sc.tl.leiden(adata,resolution=1.2)

In [ ]:
sc.tl.umap(adata,min_dist=0.4)
sc.set_figure_params(scanpy=True, dpi=200)
sc.pl.umap(adata,color=['leiden'],save='50plex_UMAP_numbers.svg')

In [ ]:
sc.pl.umap(adata,color=['Slc17a7','Plp1','Slc32a1'],save='50plex_UMAP_Scl17a7_Plp1_Slc32a1.svg')

In [ ]:
import matplotlib as plt

In [ ]:
colors = {0:"#FFFF00", 1: "#1CE6FF", 2:"#FF34FF", 3:"#FF4A46",4:"#008941",5: "#006FA6", 6:"#A30059",
    7:"#FFDBE5", 8:"#7A4900", 9:"#0000A6", 10:"#63FFAC", 11:"#B79762", 12:"#004D43", 13:"#8FB0FF", 14:"#997D87",
    15:"#5A0007", 16:"#809693", 17:"#6A3A4C", 18:"#1B4400", 19:"#4FC601", 20:"#3B5DFF", 21:"#4A3B53", 22:"#FF2F80",
    23:"#61615A", 24:"#BA0900", 25:"#6B7900", 26:"#00C2A0",27:"#6B7900", 28:"#00C2A0" ,29:"#6B7900", 30:"#00C2A0"  }

In [ ]:
cl=adata.obs['leiden'].astype(int)
cl.apply(lambda x: colors[x])

In [ ]:
import seaborn as sns
figa=plt.pyplot.scatter(x=adata.obs.X,y=adata.obs.Y,c=cl.apply(lambda x: colors[x]),s=1.5,linewidths=0, edgecolors=None)
plt.pyplot.axis('off')
plt.pyplot.savefig('E:/PROJECT_DirectRNA/Figures_revision/50plex_expression_map.svg')

In [ ]:
import matplotlib.image as mpimg
img = mpimg.imread('E:/PROJECT_DirectRNA/DAPI_20perc_50plex.tif')
from PIL import Image, ImageOps

In [ ]:
import seaborn as sns
plt.pyplot.imshow(img*20, cmap='gray',origin='lower')
plt.pyplot.scatter(x=adata.obs.Y/5,y=adata.obs.X/5,c=cl.apply(lambda x: colors[x]),s=1.5,linewidths=0, edgecolors=None)
plt.pyplot.axis('off')
plt.pyplot.savefig('E:/PROJECT_DirectRNA/Figures_revision/50plex_expression_map_DAPI.svg')

In [ ]:
sc.tl.paga(adata)
sc.pl.paga(adata, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(adata, init_pos='paga')

In [ ]:
clusters=np.unique(adata.obs.leiden)
plt.pyplot.style.use('seaborn-whitegrid')
for i in clusters:
    print(i)
    int(i)
    plt.pyplot.figure()
    plt.pyplot.imshow(img*20, cmap='gray',origin='lower')
    plt.pyplot.scatter(x=adata.obs.Y[adata.obs.leiden==i]/5,y=adata.obs.X[adata.obs.leiden==i]/5,c=cl[adata.obs.leiden==i].apply(lambda x: colors[x]),s=1.5,linewidths=0, edgecolors=None)
    plt.pyplot.axis('off')
    plt.pyplot.title('Cluster '+i)


In [ ]:
adata.var_names=expdata.columns

In [ ]:

adata.layers['scaled'] = sc.pp.scale(adata, copy=True).X

In [ ]:
annotation= {0:'OLIG 1',1:'OLIG 2',2:'ASTRO',3:'EXC 1',4:'EXC 2',5:'INH 1',6:'EXC 3',7:'INH 2',8:'ENDO',9:'Calb2+ cells',10:'EXC 4',12:'EXC 5',11:'EXC 6',13:'EXC 7',14:'EXC 8',15:'OLIG 3',
             16:'EXC 9',17:'OLIG',18:'EXC 10',19:'INH 3',20:'EXC 11',21:'EXC 12',22:'Tac2+ cells',23:'UNKNOWN',24:'C.PLEXUS/EPENDYMAL',25:'OLIG 4',26:'OLIG 5',27:'INH 4'}

In [ ]:
adata.obs['leiden']

In [ ]:
adata.obs['leiden'].astype(int).apply(lambda x: annotation[x])

In [ ]:
adata.obs['clusters2']=adata.obs['leiden'].astype(int).apply(lambda x: annotation[x])

In [ ]:
adata

In [ ]:
sc.pl.matrixplot(adata,expdata.columns, 'clusters2', dendrogram=True,colorbar_title='mean z-score',vmin=-2, vmax=2,  cmap='RdBu_r',layer='scaled',swap_axes=True)#,save='Heatmap_Expression_50plex_30cl_REV.svg')

In [ ]:

sc.pl.umap(adata,color=['clusters2'])# ,save='UMAP_50plex_withnames_revised.svg'

In [ ]:
sc.pl.umap(adata,color=['Slc17a7','Plp1','Slc32a1'],cmap='viridis',save='UMAP_50plex_SLC17A7_PLP1_SLC32A1_revised.svg')

In [ ]:
cls=adata.obs['clusters2']
#cls.apply(lambda x: colores[x])

In [ ]:
imcols

In [ ]:
imcols=np.unique(adata.obs['clusters2'])
cols=adata.uns['clusters2_colors']
colores=dict(zip(imcols,cols))

In [ ]:
colores

In [ ]:
colores

In [ ]:
cls.apply(lambda x: colores[x])

In [ ]:
import seaborn as sns
plt.imshow(img*20, cmap='gray',origin='lower')
plt.scatter(x=adata.obs.Y/5,y=adata.obs.X/5,c=cls.apply(lambda x: colores[x]),s=1.5,linewidths=0, edgecolors=None)
plt.axis('off')
plt.savefig('E:/PROJECT_DirectRNA/Figures_revision/50plex_expression_map_DAPI_clusternames.svg')

In [ ]:
subset=['EXC 1','EXC 2','EXC 3','EXC 4','EXC 5','EXC 6','EXC 7','EXC 8','EXC 9','EXC 10','EXC 11','EXC 12']

In [ ]:
clusters=subset
plt.style.use('seaborn-whitegrid')
plt.figure()
plt.imshow(img*20, cmap='gray',origin='lower')
for i in clusters:
    print(i)
    plt.scatter(x=adata.obs.Y[adata.obs['clusters2']==i]/5,y=adata.obs.X[adata.obs['clusters2']==i]/5,c=cls[adata.obs['clusters2']==i].apply(lambda x: colores[x]),s=1.5,linewidths=0, edgecolors=None)
    plt.axis('off')
    plt.title('EXCITATORY')
plt.savefig('E:/PROJECT_DirectRNA/Figures_revision/EXC_map_rev.svg')

In [ ]:
subset2=['OLIG','OLIG 1','OLIG 2','OLIG 3','OLIG 4','OLIG 5']

In [ ]:
clusters=subset2
plt.style.use('seaborn-whitegrid')
plt.figure()
plt.imshow(img*20, cmap='gray',origin='lower')
for i in clusters:
    print(i)
    plt.scatter(x=adata.obs.Y[adata.obs.clusters==i]/5,y=adata.obs.X[adata.obs.clusters==i]/5,c=cls[adata.obs.clusters==i].apply(lambda x: colores[x]),s=1.5,linewidths=0, edgecolors=None)
    plt.axis('off')
    plt.title('OLIGODENDROCYTES')
plt.savefig('E:/PROJECT_DirectRNA/Figures_revision/OLIGO_map_rev.svg')

In [ ]:
subset3=['Calb2+ cells','Tac2+ cells']

In [ ]:
clusters=subset3
plt.style.use('seaborn-whitegrid')
plt.figure()
plt.imshow(img*20, cmap='gray',origin='lower')
for i in clusters:
    print(i)
    plt.scatter(x=adata.obs.Y[adata.obs.clusters==i]/5,y=adata.obs.X[adata.obs.clusters==i]/5,c=cls[adata.obs.clusters==i].apply(lambda x: colores[x]),s=1.5,linewidths=0, edgecolors=None)
    plt.axis('off')
    plt.title('TAC2/CALB2 CELLS')
plt.savefig('E:/PROJECT_DirectRNA/Figures_revision/TAC_CALB2_cells_map_rev.svg')

In [ ]:
subset4=['INH 1','INH 2','INH 3']

In [ ]:
clusters=subset4
plt.style.use('seaborn-whitegrid')
plt.figure()
plt.imshow(img*20, cmap='gray',origin='lower')
for i in clusters:
    print(i)
    plt.scatter(x=adata.obs.Y[adata.obs.clusters==i]/5,y=adata.obs.X[adata.obs.clusters==i]/5,c=cls[adata.obs.clusters==i].apply(lambda x: colores[x]),s=1.5,linewidths=0, edgecolors=None)
    plt.axis('off')
    plt.title('INHIBITORY CELLS')
plt.savefig('E:/PROJECT_DirectRNA/Figures_revision/Inhibitory_cells_map_rev.svg')

In [ ]:
subset5=['C.PLEXUS/EPENDYMAL']

In [ ]:
clusters=subset5
plt.style.use('seaborn-whitegrid')
plt.figure()
plt.imshow(img*20, cmap='gray',origin='lower')
for i in clusters:
    print(i)
    plt.scatter(x=adata.obs.Y[adata.obs.clusters==i]/5,y=adata.obs.X[adata.obs.clusters==i]/5,c=cls[adata.obs.clusters==i].apply(lambda x: colores[x]),s=1.5,linewidths=0, edgecolors=None)
    plt.axis('off')
    plt.title('Ependymal/C.Plexus')
plt.savefig('E:/PROJECT_DirectRNA/Figures_revision/Epen_Cplexus_cells_map.svg')

In [ ]:
subset6=['microglia','endo/astro']

In [ ]:
clusters=subset6
plt.style.use('seaborn-whitegrid')
plt.figure()
plt.imshow(img*20, cmap='gray',origin='lower')
for i in clusters:
    print(i)
    plt.scatter(x=adata.obs.Y[adata.obs.clusters==i]/5,y=adata.obs.X[adata.obs.clusters==i]/5,c=cls[adata.obs.clusters==i].apply(lambda x: colores[x]),s=1.5,linewidths=0, edgecolors=None)
    plt.axis('off')
    plt.title('Microglia Endo/Astro')
plt.savefig('E:/PROJECT_DirectRNA/Figures_revision/Microglia_endo_astro_map.svg')

In [ ]:
clusters=
plt.style.use('seaborn-whitegrid')
for i in clusters:
    print(i)
    plt.figure()
    plt.imshow(img*20, cmap='gray',origin='lower')
    plt.scatter(x=adata.obs.Y[adata.obs.clusters==i]/5,y=adata.obs.X[adata.obs.clusters==i]/5,c=cls[adata.obs.clusters==i].apply(lambda x: colores[x]),s=1.5,linewidths=0, edgecolors=None)
    plt.axis('off')
    plt.title('Cluster '+i)

In [ ]:
clusters=np.unique(adata.obs.clusters)
plt.style.use('seaborn-whitegrid')
for i in clusters:
    print(i)
    plt.figure()
    plt.imshow(img*20, cmap='gray',origin='lower')
    plt.scatter(x=adata.obs.Y[adata.obs.clusters==i]/5,y=adata.obs.X[adata.obs.clusters==i]/5,c=cls[adata.obs.clusters==i].apply(lambda x: colores[x]),s=1.5,linewidths=0, edgecolors=None)
    plt.axis('off')
    plt.title('Cluster '+i)

In [ ]:
datafra=pd.DataFrame(data=[adata.obs.X,adata.obs.Y,adata.obs.leiden,adata.obs.clusters])

In [ ]:
datafra2=datafra.transpose()
datafra2.to_csv('E:\PROJECT_DirectRNA\directRNA\clusters_leiden_50X.csv')
datafra2

In [ ]:
ax = sc.pl.heatmap(adata,expdata.columns,groupby= 'clusters', layer='scaled', vmin=-2, vmax=2, cmap='RdBu_r', dendrogram=True, swap_axes=True)

In [ ]:
adata.write('E:/PROJECT_DirectRNA/Figures_revision/50plex_analysis_REV.h5')

# SAVING INFO

In [ ]:
import scanpy as sc
adata=sc.read_h5ad('G:/PROJECT_DirectRNA/Figures_revision/MAIN_50plex_clustering/50plex_analysis.h5')

In [ ]:
SUPPTABLE=pd.DataFrame(adata.X)

In [ ]:
SUPPTABLE.columns=adata.var.index

In [ ]:
SUPPTABLE

In [ ]:
SUPPTABLE['X']=list(adata.obs['X'])
SUPPTABLE['Y']=list(adata.obs['Y'])

In [ ]:
SUPPTABLE['Main_cluster']=list(adata.obs['clusters'])
SUPPTABLE['Number']=list(adata.obs['Number'])

In [ ]:
SUPPTABLE.to_csv('G:/PROJECT_DirectRNA/Figures_revision/New_supplementary_info/supp_expression_cells.csv')

In [ ]:
res = pd.DataFrame(columns=adata.var_names, index=adata.obs['clusters'].cat.categories)                                                                                                 

for clust in adata.obs.clusters.cat.categories: 
    res.loc[clust] = adata[adata.obs['clusters'].isin([clust]),:].X.mean(0)

In [ ]:
res.to_csv('G:/PROJECT_DirectRNA/Figures_revision/New_supplementary_info/supp_mean_expression_celltypes.csv')

# SPAGE INTEGRATION

Next, we integrate the different datasets using SPAGE

In [ ]:
import numpy as np
import pandas as pd
import loompy
import matplotlib.pyplot as plt
import scipy.stats as st
#from SpaGE.main import SpaGE
import warnings
warnings.filterwarnings('ignore')

In [ ]:
ds = loompy.connect('U:/PROJECT_DirectRNA/l5_all (1).loom')
d = {'CellID': ds.ca.CellID, 'Age': ds.ca.Age,'Cluster':ds.ca.ClusterName,'Region':ds.ca.Region}
RNA_metaX=pd.DataFrame(data=d)
#RNA_meta.columns=['Age','ID']

RNA_metaX
RNA_metaX['BigCluster']=ds.ca.TaxonomyRank4
select=RNA_metaX['Region'].isin(['Cortex','Brain', 'CNS','Hippocampus', 'Hippocampus,Cortex','Telencephalon','Thalamus'])
#RNA_data.shape
RNA_meta=RNA_metaX[select]


In [ ]:
RNA=pd.DataFrame(ds[:,:])
RNA.index=ds.ra.Gene
RNA_data = RNA

In [ ]:
RNA_data

In [ ]:
# filter lowely expressed genes
RNA_data=RNA_data.loc[:,select]
Genes_count = np.sum(RNA_data > 0, axis=1)
RNA_data = RNA_data.loc[Genes_count >=10,:]
del Genes_count

# Normalization
def Log_Norm_cpm(x):
    return np.log(((x/np.sum(x))*1000000) + 1)


In [ ]:
expression=RNA_data

In [ ]:
ge=['Mbp', 'Slc17a7', 'Lamp5', 'Cd24a', 'Gad2', 'Cnr1', 'Pthlh', 'Crhbp',
       'Slc32a1', 'Vip', 'Tbr1', 'Kcnip2', 'Crh', 'Cpne5', 'Rorb', 'Serpinf1',
       'Gfap', 'Aldoc', 'Syt6', 'Mfge8', 'Pdgfra', 'Plp1', 'Sox10',
       'Itpr2', 'Hexb', 'Anln', 'Bmp4', 'Ctps', 'Mrc1', 'Acta2', 'Apln',
       'Flt1', 'Vtn', 'Ttr', 'Synpr', 'Npy', 'Sst', 'Foxj1', 'Pcp4', 'Calm2',
       'Gad1', 'Pvalb', 'Penk', 'Rgs4', 'Nrn1', 'Tac2', 'Calb2', 'Slc6a1',
       'Enc1']

In [ ]:

expression=expression.loc[ge,:]

In [ ]:
expression=expression.transpose()

# Load Spatial

In [ ]:
osmFISH_meta=adata.obs
#osmFISH_meta.index=ds.index
osmFISH_data=adata.X
osmFISH_data=pd.DataFrame(osmFISH_data)
osmFISH_data=pd.DataFrame.transpose(osmFISH_data)
osmFISH_data

In [ ]:
osmFISH_data.index=adata.var_names

In [ ]:
# Normalization
cell_count = np.sum(osmFISH_data,axis=0)
def Log_Norm_spatial(x):
    return np.log(((x/np.sum(x))*np.median(cell_count)) + 1)
osmFISH_data = osmFISH_data.apply(Log_Norm_spatial,axis=0)

In [ ]:
sp_df = pd.DataFrame.transpose(osmFISH_data)
sc_df = pd.DataFrame.transpose(RNA_data)

In [ ]:
from SpaGE.principal_vectors import PVComputation
from sklearn.neighbors import NearestNeighbors
# Gene_set = ['Lamp5','Kcnip2','Rorb','Tbr1','Syt6']
i = 'Lamp5'
sp_df = sp_df.drop(i,axis=1)
genes_to_predict = [i]
# genes_to_predict = np.setdiff1d(sc_df.columns,sp_df.columns)
n_pv = 30

RNA_data_scaled = pd.DataFrame(data=st.zscore(sc_df,axis=0),
                               index = sc_df.index,columns=sc_df.columns)
Spatial_data_scaled = pd.DataFrame(data=st.zscore(sp_df,axis=0),
                               index = sp_df.index,columns=sp_df.columns)
Common_data = RNA_data_scaled[np.intersect1d(Spatial_data_scaled.columns,RNA_data_scaled.columns)]

pv_FISH_RNA = PVComputation(
        n_factors = n_pv,
        n_pv = n_pv,
        dim_reduction = 'pca',
        dim_reduction_target = 'pca'
)

In [ ]:
from tqdm import tqdm
pv_FISH_RNA.fit(Common_data,Spatial_data_scaled[Common_data.columns])
S = pv_FISH_RNA.source_components_.T
Effective_n_pv = sum(np.diag(pv_FISH_RNA.cosine_similarity_matrix_) > 0.3)
S = S[:,0:Effective_n_pv]
Common_data_t = Common_data.dot(S)
FISH_exp_t = Spatial_data_scaled[Common_data.columns].dot(S)
nbrs = NearestNeighbors(n_neighbors=15, algorithm='auto',
                        metric = 'cosine').fit(Common_data_t)
distances, indices = nbrs.kneighbors(FISH_exp_t)

# Infer gene expression of genes not in panel on subsample of spots
n_sub = 1000
r_idx_sub = np.random.choice(sp_df.shape[0], 1000, replace=False)
Imp_New_Genes = pd.DataFrame(np.zeros((n_sub,len(genes_to_predict))),
                             columns=genes_to_predict, index = r_idx_sub)
for j in tqdm(r_idx_sub):
    weights = 1-(distances[j,:][distances[j,:]<1])/(np.sum(distances[j,:][distances[j,:]<1]))
    weights = weights/(len(weights)-1)
    Imp_New_Genes.loc[j,:] = np.dot(weights,sc_df[genes_to_predict].iloc[indices[j,:][distances[j,:] < 1]])

In [ ]:
fig, ax2 = plt.subplots(1,1)
# ax1.axis('off')
# cmap =sp_df.loc[:,i].values
# cmap[cmap > np.percentile(cmap,99)] = np.percentile(cmap,99)
# ax1.scatter(adata_sp_nn.obs.loc[sp_df.index,'spotX'],adata_sp_nn.obs.loc[sp_df.index,'spotY'],s=1,c=cmap)
# ax1.set_title('Measured ' + i, fontsize = 12)
# ax1.set_ylabel(i)

ax2.axis('off')
cmap = Imp_New_Genes.loc[:,i].values
cmap[cmap > np.percentile(cmap,99)] = np.percentile(cmap,99)
ax2.scatter(osmFISH_meta.iloc[r_idx_sub,0],osmFISH_meta.iloc[r_idx_sub,1],s=10,c=cmap)
ax2.set_title('Predicted ' + i, fontsize = 12)
plt.axis('scaled')
plt.gca().invert_yaxis()
# plt.gca().invert_xaxis()

In [ ]:
from SpaGE.principal_vectors import PVComputation
from sklearn.neighbors import NearestNeighbors

# Gene_set = ['Lamp5','Kcnip2','Rorb','Tbr1','Syt6']
# i = 'SLC39A8'
# sp_df = sp_df.drop(i,axis=1)
# genes_to_predict = [i]

genes_to_predict = np.setdiff1d(sc_df.columns,sp_df.columns)
n_pv = 30

RNA_data_scaled = pd.DataFrame(data=st.zscore(sc_df,axis=0),
                               index = sc_df.index,columns=sc_df.columns)
Spatial_data_scaled = pd.DataFrame(data=st.zscore(sp_df,axis=0),
                               index = sp_df.index,columns=sp_df.columns)
Common_data = Spatial_data_scaled[np.intersect1d(Spatial_data_scaled.columns,RNA_data_scaled.columns)]


pv_SC_RNA = PVComputation(
        n_factors = n_pv,
        n_pv = n_pv,
        dim_reduction = 'pca',
        dim_reduction_target = 'pca'
)

In [ ]:
osmFISH_meta['Cluster']=osmFISH_meta['leiden']

In [ ]:
pv_SC_RNA.fit(Common_data,RNA_data_scaled[Common_data.columns])

S = pv_SC_RNA.source_components_.T

Effective_n_pv = sum(np.diag(pv_SC_RNA.cosine_similarity_matrix_) > 0.3)
S = S[:,0:Effective_n_pv]

Common_data_t = Common_data.dot(S)
SC_exp_t = RNA_data_scaled[Common_data.columns].dot(S)

nbrs = NearestNeighbors(n_neighbors=100, algorithm='auto',
                        metric = 'cosine').fit(Common_data_t)
distances, indices = nbrs.kneighbors(SC_exp_t)

sc_cluster_df = pd.DataFrame(np.zeros((sc_df.shape[0],len(osmFISH_meta.Cluster.unique()))), index=sc_df.index, columns=np.arange(len(osmFISH_meta.Cluster.unique())))


In [ ]:
for j in tqdm(range(0,sc_df.shape[0])):
    weights = 1-(distances[j,:][distances[j,:]<1])/(np.sum(distances[j,:][distances[j,:]<1]))
    weights = weights/(len(weights)-1)
    for c in osmFISH_meta.iloc[indices[j,:],:].Cluster.unique():
        c_idx = np.where(osmFISH_meta.iloc[indices[j,:],:].Cluster.values==c)[0]
        sc_cluster_df.iloc[j,int(c)] = np.sum(weights[c_idx])

In [ ]:
sc_cluster_df.to_pickle('E:/PROJECT_DirectRNA/Figures_revision/ScRNAseq_integration/sc_cluster_df_dRNA_REV.h5')

In [ ]:
sc_cluster_df = pd.read_pickle('G:/PROJECT_DirectRNA/Figures_revision/ScRNAseq_integration/sc_cluster_df_dRNA_REV.h5')

In [ ]:
sc_cluster_df

In [ ]:
cols=sc_cluster_df.columns

In [ ]:
colo=pd.DataFrame(cols,columns=['col'])

In [ ]:
annotation= {0:'OLIG 1',1:'OLIG 2',2:'ASTRO',3:'EXC 1',4:'EXC 2',5:'INH 1',6:'EXC 3',7:'INH 2',8:'ENDO',9:'Calb2+ cells',10:'EXC 4',12:'EXC 5',11:'EXC 6',13:'EXC 7',14:'EXC 8',15:'OLIG 3',
             16:'EXC 9',17:'OLIG',18:'EXC 10',19:'INH 3',20:'EXC 11',21:'EXC 12',22:'Tac2+ cells',23:'UNKNOWN',24:'C.PLEXUS/EPENDYMAL',25:'OLIG 4',26:'OLIG 5',27:'INH 4'}
sc_cluster_df.columns=colo.col.apply(lambda x: annotation[x])

In [ ]:

import seaborn as sns
ct_color_d = {0: '#F0756D',
            1: '#E18725',
            2: '#C5972B',
            3: '#9EA237',
            4: '#6AAF43',
            5: '#29B14A',
            6: '#30B47B',
            7: '#23B9AA',
            8: '#0AB9D6',
            9: '#37AADF',
            10: '#6F93CA',
            11: '#9D84BB',
            12: '#BB76B0',
            13: '#D96DA8',
            14: '#EE669E',
             15: '#F0756D',
            16: '#E18725',
            17: '#C5972B',
            18: '#9EA237',
            19: '#F0756D',
            20: '#E18725',
            21: '#C5972B',
            22: '#9EA237',
            23: '#6AAF43',
            24: '#29B14A',
            25: '#30B47B',
            26: '#23B9AA',
            27: '#0AB9D6',
            28: '#0AB9D6',
          }

row_colors = RNA_meta.loc[:,'Cluster'].map(ct_color_d)

sns.clustermap(sc_cluster_df.loc[RNA_meta.sort_values('Cluster').index,:], row_cluster=True, method='average', figsize=(20,20),standard_scale=1,row_colors= row_colors) # 

In [ ]:

d = {}
for col in sc_cluster_df.columns:
    sc_c = []
    for c in np.sort(RNA_meta.BigCluster.unique()):
        sc_c.append(sc_cluster_df.loc[RNA_meta[(RNA_meta.BigCluster==c)].index,col].sum())
    if np.array(sc_c).argmax() in d:
        d[np.array(sc_c).argmax()].append(col)
    else:
        d[np.array(sc_c).argmax()] = [col]

In [ ]:
col_order = sum(list({k: d[k] for k in np.sort(list(d.keys()))}.values()),[])


In [ ]:
ct_color_d = {'Oligodendrocytes': '#F0756D',
           'Telencephalon projecting excitatory neurons': '#E18725',
            'Telencephalon inhibitory interneurons': '#C5972B',
            'Di- and mesencephalon excitatory neurons': '#9EA237',
            'Non-glutamatergic neuroblasts': '#6AAF43',
            'Oligodendrocyte precursor cells': '#29B14A',
            'Peptidergic neurons': '#30B47B',
            'Vascular and leptomeningeal cells': '#23B9AA',
            'Vascular smooth muscle cells': '#0AB9D6',
            'Pericytes': '#37AADF',
            'Vascular endothelial cells': '#6F93CA',
            'Microglia': '#9D84BB',
            'Perivascular macrophages': '#BB76B0',
           'Astrocytes': '#D96DA8',
           'Ependymal cells': '#EE669E'
          }
    

In [ ]:
exc=sc_cluster_df.idxmax(axis=1)
new_idx = []
for c in np.sort(RNA_meta.BigCluster.unique()):
#     Z = scipy.cluster.hierarchy.linkage(adata_tmp[adata_tmp.obs.cluster==c].X, method='single', metric='cosine')
#     new_idx.append(adata_tmp[adata_tmp.obs.cluster==c].obs.iloc[np.flip(scipy.cluster.hierarchy.leaves_list(Z)),:].index.values)
   for sp_c in col_order:
        new_idx.append(RNA_meta[(exc==sp_c) & (RNA_meta.BigCluster==c)].index.values)
      
new_idx = np.concatenate(new_idx)
inde=RNA_meta['BigCluster']

In [ ]:

row_colors = inde[new_idx].map(ct_color_d)

In [ ]:
import collections

a = [1,1,1,1,2,2,2,2,3,3,4,5,5]
counter=collections.Counter(RNA_meta.BigCluster)

In [ ]:
np.sort(RNA_meta.BigCluster.unique())

In [ ]:
cols=pd.DataFrame(sc_cluster_df.columns,columns=['col'])

In [ ]:
annotation= {23:'OPC',0:'Oligo 1',22:'Oligo 2',
26:'Oligo 3',3:'Oligo 4',9:'Oligo 5',29:'Oligo 6',7:'Inhibitory 1',1:'Inhibitory 2'
             ,20:'Inhibitory 3',21:'Tac2+ cells',25:'Unknown 1',6:'endo/astro',11:'endo/astro',28:'microglia'
             ,15:'Unknown 2',24:'Epen/C.plexus',8:'Calb2+ cells',14:'EXC1',10:'EXC2',12:'EXC3',
18:'EXC4',27:'EXC5',17:'EXC6',19:'EXC7',
2:'EXC8',26:'Oligo3',13:'EXC10',4:'EXC11',5:'EXC12',16:'EXC13'}
sc_cluster_df.columns=cols.col.apply(lambda x: annotation[x])

In [ ]:
sns_plot=sns.clustermap(sc_cluster_df.loc[new_idx,col_order],col_cluster=False, row_cluster=False,metric='cosine', method='ward', figsize=(20,20),row_colors= row_colors) #

In [ ]:
sns_plot.savefig("F:/PROJECT_DirectRNA/Figures_revision/ScRNAseq_integration/SpaGE_Single_cell_DRNA_correspondence_dpi700.png",dpi=700)

In [ ]:

d = {}
for col in sc_cluster_df.columns:
    sc_c = []
    for c in np.sort(adata_tmp.obs.cluster.unique()):
        sc_c.append(sc_cluster_df.loc[adata_tmp[(adata_tmp.obs.cluster==c)].obs.index,col].sum())
    if np.array(sc_c).argmax() in d:
        d[np.array(sc_c).argmax()].append(col)
    else:
        d[np.array(sc_c).argmax()] = [col]